Oct 30, 2019

Feasability test for Sandy Hook in /r/politics. 

In [8]:
from tqdm import tqdm
import os
os.chdir('../../')
from convokit import Corpus, User, Utterance

In [9]:
corpus = Corpus(filename='/Users/calebchiam/Documents/politics-filtered-corpus') #1.96 GB, don't duplicate.

In [10]:
corpus.print_summary_stats()

Number of Users: 289890
Number of Utterances: 3237456
Number of Conversations: 94768


In [11]:
import nltk

In [12]:
shooting_timestamps = {
 'Fort Hood shooting': (1257397200, 1258088400),
 'Binghamton shootings': (1238731200, 1239422400),
 'Geneva County massacre': (1236657600, 1237348800),
 'Sandy Hook Elementary School shooting': (1355461200, 1356152400),
 'Aurora theater shooting': (1342756800, 1343448000),
 'Washington Navy Yard shooting': (1379304000, 1379995200),
 'San Bernardino attack': (1449032400, 1449723600),
 'Orlando nightclub shooting': (1465704000, 1466395200),
 'Las Vegas shooting': (1506830400, 1507521600),
 'Sutherland Springs church shooting': (1509854400, 1510545600),
 'Stoneman Douglas High School shooting': (1518584400, 1519275600),
 'Santa Fe High School shooting': (1526616000, 1527307200)}

# 'Virginia Tech shooting': (1176696000, 1177387200),
#  'Thousand Oaks shooting': (1541566800, 1542258000),
# 'Pittsburgh synagogue shooting': (1540612800, 1541304000)

In [13]:
def identify_timestamp_category(timestamp):
    for k, v in shooting_timestamps.items():
        if v[0] <= timestamp <= v[1]:
            return k

## Label conversations and utterances by event time category

In [14]:
for convo in corpus.iter_conversations():
    convo.meta['event'] = identify_timestamp_category(convo.meta['timestamp'])
    for utt in convo.iter_utterances():
        utt.meta['event'] = convo.meta['event']

## Tokenize titles with lemmatization

In [15]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('english')

In [16]:
convo = next(corpus.iter_conversations())

In [17]:
def tokenize_and_stem(sentence):
    return set(stemmer.stem(word) for word in nltk.word_tokenize(sentence))

In [18]:
for convo in corpus.iter_conversations():
    convo.meta['stem_tokens'] = tokenize_and_stem(convo.meta['title'])

In [19]:
generic_words = {stemmer.stem(word) for word in ['shooting', 'shot', 'deaths', 'died', 'injured', 'guns', 'killing', 'attack', 'massacre', 'victim']}

## Generate specific words dictionaries for each event

In [20]:
specific_words_raw = {
    'Fort Hood shooting': ['fort hood', "nidal hasan"],
    'Binghamton shootings': ['binghamton', "jiverly", 'antares', 'wong'],
    'Geneva County massacre': ['geneva', "county", 'mclendon'],
 'Sandy Hook Elementary School shooting': ['sandy hook', "adam lanza"],
 'Aurora theater shooting': ['aurora theater', 'james eagan holmes', 'colorado'],
 'Washington Navy Yard shooting': ['washington navy yard', 'aaron alexis', 'navy yard'],
 'San Bernardino attack': ['syed rizwan farook', 'tashfeen malik', 'san bernandino', 'san bernandino'],
 'Orlando nightclub shooting': ['omar mateen'],
 'Las Vegas shooting': ['las vegas shooting', 'stephen paddock'],
 'Sutherland Springs church shooting': ['sutherland springs', 'sutherland church', 'devin patrick kelley'],
 'Stoneman Douglas High School shooting': ['Marjory Stoneman Douglas High School', 'Parkland', 'Florida'],
 'Santa Fe High School shooting': ['santa fe high school', 'dimitrios pagourtzis']}


In [21]:
specific_words_list = {}
for k, v in specific_words_raw.items():
    specific_words_list[k] = []
    for phrase in v + [k]:
        specific_words_list[k].extend(phrase.split())
    specific_words_list[k] = set([stemmer.stem(w) for w in set(specific_words_list[k])])
    specific_words_list[k] -= generic_words

## Label conversations and utterances with whether they are *actually* associated with the event

In [22]:
for convo in corpus.iter_conversations():
    event = convo.meta['event']
    tokens = convo.meta['stem_tokens']
    
    if len(tokens.intersection(generic_words)) > 0 or len(tokens.intersection(specific_words_list.get(event, {}))) > 0:
        convo.meta['valid'] = True
    else:
        convo.meta['valid'] = False
        
    for utt in convo.iter_utterances():
        utt.meta['valid'] = convo.meta['valid']

## Let's see a distribution of the counts

In [23]:
from collections import defaultdict
convo_counts = defaultdict(int)
utt_counts = defaultdict(int)
for convo in corpus.iter_conversations():
    if convo.meta['valid']: convo_counts[convo.meta['event']] += 1

for utt in corpus.iter_utterances():
    if utt.meta['valid']: utt_counts[utt.meta['event']] += 1
    

In [24]:
convo_counts

defaultdict(int,
            {'San Bernardino attack': 887,
             'Orlando nightclub shooting': 1770,
             'Sandy Hook Elementary School shooting': 3167,
             'Stoneman Douglas High School shooting': 2674,
             'Las Vegas shooting': 1729,
             'Sutherland Springs church shooting': 811,
             'Geneva County massacre': 181,
             'Washington Navy Yard shooting': 483,
             'Fort Hood shooting': 513,
             'Aurora theater shooting': 1461,
             None: 1055,
             'Binghamton shootings': 186,
             'Santa Fe High School shooting': 808})

In [25]:
utt_counts

defaultdict(int,
            {'San Bernardino attack': 42854,
             'Orlando nightclub shooting': 87024,
             'Sandy Hook Elementary School shooting': 84810,
             'Stoneman Douglas High School shooting': 218585,
             'Las Vegas shooting': 97896,
             'Sutherland Springs church shooting': 48353,
             'Geneva County massacre': 1523,
             'Washington Navy Yard shooting': 15286,
             'Fort Hood shooting': 6154,
             'Aurora theater shooting': 44642,
             None: 37173,
             'Binghamton shootings': 2047,
             'Santa Fe High School shooting': 80168})

In [ ]:
from tqdm import tqdm

In [31]:
for utt in tqdm(list(corpus.iter_utterances())):
    if "stem_tokens" in utt.meta: continue
    if utt.meta['valid']:
        utt.meta['stem_tokens'] = tokenize_and_stem(utt.text)
    else:
        utt.meta['stem_tokens'] = None




  0%|          | 0/3237456 [00:00<?, ?it/s]


  3%|▎         | 102488/3237456 [00:00<00:03, 1024868.76it/s]


  7%|▋         | 227484/3237456 [00:00<00:02, 1083395.42it/s]


 11%|█         | 350353/3237456 [00:00<00:02, 1123239.63it/s]


 15%|█▍        | 478027/3237456 [00:00<00:02, 1165265.66it/s]


 18%|█▊        | 595847/3237456 [00:00<00:02, 1169078.21it/s]


 22%|██▏       | 696307/3237456 [00:01<00:12, 207403.71it/s] 


 24%|██▎       | 768760/3237456 [00:03<00:19, 123733.53it/s]


 25%|██▌       | 821389/3237456 [00:03<00:16, 145054.86it/s]


 29%|██▉       | 940699/3237456 [00:03<00:11, 196958.60it/s]


 31%|███▏      | 1019725/3237456 [00:03<00:08, 253199.87it/s]


 34%|███▎      | 1089160/3237456 [00:10<01:08, 31139.48it/s] 


 35%|███▍      | 1125874/3237456 [00:21<03:57, 8880.06it/s] 


 35%|███▍      | 1125874/3237456 [00:21<03:57, 8880.06it/s]


 35%|███▍      | 1126207/3237456 [00:21<11:41, 3008.18it/s]


 35%|███▍      | 1126476/3237456 [00:21<14:23, 2445.01it/s]


 

 35%|███▌      | 1140815/3237456 [00:32<22:11, 1574.64it/s]


 35%|███▌      | 1140975/3237456 [00:32<24:14, 1441.53it/s]


 35%|███▌      | 1141138/3237456 [00:32<23:26, 1490.27it/s]


 35%|███▌      | 1141328/3237456 [00:33<21:56, 1592.58it/s]


 35%|███▌      | 1141505/3237456 [00:33<21:18, 1639.17it/s]


 35%|███▌      | 1141698/3237456 [00:33<20:23, 1713.49it/s]


 35%|███▌      | 1141873/3237456 [00:33<20:30, 1702.70it/s]


 35%|███▌      | 1142046/3237456 [00:33<21:53, 1595.73it/s]


 35%|███▌      | 1142228/3237456 [00:33<21:05, 1656.29it/s]


 35%|███▌      | 1142428/3237456 [00:33<20:01, 1743.23it/s]


 35%|███▌      | 1142606/3237456 [00:33<20:14, 1725.17it/s]


 35%|███▌      | 1142781/3237456 [00:33<20:19, 1717.90it/s]


 35%|███▌      | 1142955/3237456 [00:34<20:39, 1689.69it/s]


 35%|███▌      | 1143126/3237456 [00:34<20:47, 1679.45it/s]


 35%|███▌      | 1143349/3237456 [00:34<19:14, 1813.58it/s]


 35%|███▌      | 1143535/3237456 [00:34<19:18, 1807.23it/s]


 35%|███

 36%|███▌      | 1161449/3237456 [00:45<23:46, 1455.43it/s]


 36%|███▌      | 1161606/3237456 [00:45<23:17, 1485.07it/s]


 36%|███▌      | 1161777/3237456 [00:45<22:23, 1544.75it/s]


 36%|███▌      | 1161942/3237456 [00:45<22:00, 1571.59it/s]


 36%|███▌      | 1162108/3237456 [00:45<21:44, 1590.58it/s]


 36%|███▌      | 1162269/3237456 [00:45<22:17, 1551.54it/s]


 36%|███▌      | 1162426/3237456 [00:45<22:43, 1521.43it/s]


 36%|███▌      | 1162579/3237456 [00:45<23:45, 1455.96it/s]


 36%|███▌      | 1162751/3237456 [00:46<22:40, 1525.32it/s]


 36%|███▌      | 1162916/3237456 [00:46<22:30, 1536.56it/s]


 36%|███▌      | 1163071/3237456 [00:46<29:15, 1181.35it/s]


 36%|███▌      | 1163232/3237456 [00:46<26:59, 1280.99it/s]


 36%|███▌      | 1163372/3237456 [00:46<26:53, 1285.66it/s]


 36%|███▌      | 1163532/3237456 [00:46<25:21, 1363.09it/s]


 36%|███▌      | 1163676/3237456 [00:46<25:11, 1372.27it/s]


 36%|███▌      | 1163836/3237456 [00:46<24:10, 1429.34it/s]


 36%|███

 36%|███▋      | 1179943/3237456 [00:57<20:35, 1664.67it/s]


 36%|███▋      | 1180166/3237456 [00:57<19:02, 1800.17it/s]


 36%|███▋      | 1180372/3237456 [00:58<18:28, 1855.27it/s]


 36%|███▋      | 1180628/3237456 [00:58<17:04, 2007.49it/s]


 36%|███▋      | 1180837/3237456 [00:58<17:31, 1956.80it/s]


 36%|███▋      | 1181038/3237456 [00:58<17:51, 1919.99it/s]


 36%|███▋      | 1181237/3237456 [00:58<17:40, 1938.67it/s]


 36%|███▋      | 1181434/3237456 [00:58<19:38, 1745.13it/s]


 36%|███▋      | 1181640/3237456 [00:58<18:46, 1825.21it/s]


 37%|███▋      | 1181877/3237456 [00:58<17:28, 1959.72it/s]


 37%|███▋      | 1182087/3237456 [00:58<17:10, 1994.52it/s]


 37%|███▋      | 1182291/3237456 [00:58<17:20, 1975.82it/s]


 37%|███▋      | 1182502/3237456 [00:59<17:00, 2013.30it/s]


 37%|███▋      | 1182706/3237456 [00:59<17:36, 1945.44it/s]


 37%|███▋      | 1182903/3237456 [00:59<18:51, 1815.82it/s]


 37%|███▋      | 1183126/3237456 [00:59<17:50, 1919.55it/s]


 37%|███

 37%|███▋      | 1205450/3237456 [01:10<11:29, 2948.30it/s]


 37%|███▋      | 1205748/3237456 [01:10<11:35, 2919.27it/s]


 37%|███▋      | 1206162/3237456 [01:10<10:34, 3200.79it/s]


 37%|███▋      | 1206538/3237456 [01:10<10:10, 3329.20it/s]


 37%|███▋      | 1206879/3237456 [01:10<10:29, 3226.12it/s]


 37%|███▋      | 1207279/3237456 [01:11<09:53, 3422.79it/s]


 37%|███▋      | 1207758/3237456 [01:11<09:02, 3742.16it/s]


 37%|███▋      | 1208147/3237456 [01:11<08:59, 3760.33it/s]


 37%|███▋      | 1208534/3237456 [01:11<09:40, 3495.09it/s]


 37%|███▋      | 1209003/3237456 [01:11<08:58, 3767.23it/s]


 37%|███▋      | 1209394/3237456 [01:11<09:01, 3743.05it/s]


 37%|███▋      | 1209807/3237456 [01:11<08:51, 3816.75it/s]


 37%|███▋      | 1210197/3237456 [01:11<08:49, 3829.78it/s]


 37%|███▋      | 1210585/3237456 [01:11<08:49, 3828.36it/s]


 37%|███▋      | 1210987/3237456 [01:12<08:58, 3765.55it/s]


 37%|███▋      | 1211424/3237456 [01:12<08:37, 3913.74it/s]


 37%|███

...

 82%|████████▏ | 2646011/3237456 [16:02<01:17, 7598.48it/s]


 82%|████████▏ | 2646868/3237456 [16:03<01:15, 7855.09it/s]


 82%|████████▏ | 2647663/3237456 [16:03<01:15, 7817.98it/s]


 82%|████████▏ | 2648472/3237456 [16:03<01:15, 7803.14it/s]


 82%|████████▏ | 2649548/3237456 [16:03<01:09, 8457.67it/s]


 82%|████████▏ | 2650413/3237456 [16:03<01:24, 6963.21it/s]


 82%|████████▏ | 2651167/3237456 [16:03<01:26, 6767.14it/s]


 82%|████████▏ | 2651916/3237456 [16:03<01:24, 6951.14it/s]


 82%|████████▏ | 2652642/3237456 [16:03<01:24, 6922.23it/s]


 82%|████████▏ | 2653416/3237456 [16:03<01:22, 7117.70it/s]


 82%|████████▏ | 2654181/3237456 [16:04<01:20, 7232.09it/s]


 82%|████████▏ | 2654916/3237456 [16:04<01:20, 7242.02it/s]


 82%|████████▏ | 2655671/3237456 [16:04<01:19, 7317.99it/s]


 82%|████████▏ | 2656540/3237456 [16:04<01:16, 7640.02it/s]


 82%|████████▏ | 2657585/3237456 [16:04<01:10, 8282.34it/s]


 82%|████████▏ | 2658435/3237456 [16:04<01:14, 7773.24it/s]


 82%|███

 84%|████████▍ | 2735254/3237456 [16:16<00:46, 10699.85it/s]


 85%|████████▍ | 2736382/3237456 [16:16<00:52, 9564.59it/s] 


 85%|████████▍ | 2737476/3237456 [16:16<00:50, 9866.86it/s]


 85%|████████▍ | 2738893/3237456 [16:16<00:46, 10743.92it/s]


 85%|████████▍ | 2740240/3237456 [16:16<00:43, 11438.33it/s]


 85%|████████▍ | 2741444/3237456 [16:16<00:43, 11279.85it/s]


 85%|████████▍ | 2742609/3237456 [16:16<00:46, 10618.31it/s]


 85%|████████▍ | 2743705/3237456 [16:16<00:46, 10645.51it/s]


 85%|████████▍ | 2744793/3237456 [16:16<00:50, 9814.74it/s] 


 85%|████████▍ | 2745957/3237456 [16:17<00:47, 10295.88it/s]


 85%|████████▍ | 2747634/3237456 [16:17<00:42, 11636.84it/s]


 85%|████████▍ | 2748892/3237456 [16:17<00:41, 11902.70it/s]


 85%|████████▍ | 2750834/3237456 [16:17<00:36, 13465.83it/s]


 85%|████████▌ | 2752289/3237456 [16:17<00:35, 13716.55it/s]


 85%|████████▌ | 2754071/3237456 [16:17<00:32, 14718.99it/s]


 85%|████████▌ | 2757460/3237456 [16:17<00:27, 17712.15i

 89%|████████▉ | 2887528/3237456 [16:29<00:24, 14014.12it/s]


 89%|████████▉ | 2889019/3237456 [16:29<00:24, 13978.67it/s]


 89%|████████▉ | 2890480/3237456 [16:30<00:29, 11920.92it/s]


 89%|████████▉ | 2891769/3237456 [16:30<00:36, 9538.59it/s] 


 89%|████████▉ | 2892867/3237456 [16:30<00:35, 9632.29it/s]


 89%|████████▉ | 2893931/3237456 [16:30<00:34, 9864.20it/s]


 89%|████████▉ | 2894990/3237456 [16:30<00:34, 9895.11it/s]


 89%|████████▉ | 2896030/3237456 [16:30<00:41, 8137.64it/s]


 89%|████████▉ | 2896931/3237456 [16:30<00:41, 8110.59it/s]


 90%|████████▉ | 2897803/3237456 [16:30<00:41, 8106.84it/s]


 90%|████████▉ | 2898706/3237456 [16:31<00:41, 8227.90it/s]


 90%|████████▉ | 2899777/3237456 [16:31<00:38, 8838.71it/s]


 90%|████████▉ | 2900695/3237456 [16:31<00:39, 8601.71it/s]


 90%|████████▉ | 2901850/3237456 [16:31<00:36, 9298.26it/s]


 90%|████████▉ | 2903216/3237456 [16:31<00:35, 9415.30it/s]


 90%|████████▉ | 2904754/3237456 [16:31<00:31, 10634.90it/s]


 90

 93%|█████████▎| 2997798/3237456 [16:43<01:09, 3467.20it/s]


 93%|█████████▎| 2998228/3237456 [16:43<01:05, 3625.20it/s]


 93%|█████████▎| 2998727/3237456 [16:43<01:00, 3946.55it/s]


 93%|█████████▎| 2999133/3237456 [16:43<01:02, 3840.05it/s]


 93%|█████████▎| 2999725/3237456 [16:43<00:55, 4289.20it/s]


 93%|█████████▎| 3000511/3237456 [16:43<00:47, 4965.54it/s]


 93%|█████████▎| 3001192/3237456 [16:43<00:43, 5396.24it/s]


 93%|█████████▎| 3001828/3237456 [16:44<00:41, 5646.75it/s]


 93%|█████████▎| 3002454/3237456 [16:44<00:40, 5793.46it/s]


 93%|█████████▎| 3003181/3237456 [16:44<00:38, 6157.42it/s]


 93%|█████████▎| 3003937/3237456 [16:44<00:35, 6489.26it/s]


 93%|█████████▎| 3004704/3237456 [16:44<00:34, 6800.48it/s]


 93%|█████████▎| 3005616/3237456 [16:44<00:31, 7358.76it/s]


 93%|█████████▎| 3006741/3237456 [16:44<00:28, 8201.46it/s]


 93%|█████████▎| 3007762/3237456 [16:44<00:26, 8713.94it/s]


 93%|█████████▎| 3008677/3237456 [16:44<00:25, 8801.43it/s]


 93%|███

 97%|█████████▋| 3132753/3237456 [16:56<00:07, 13913.50it/s]


 97%|█████████▋| 3134486/3237456 [16:56<00:06, 14771.25it/s]


 97%|█████████▋| 3136333/3237456 [16:56<00:06, 15707.53it/s]


 97%|█████████▋| 3138245/3237456 [16:56<00:05, 16568.45it/s]


 97%|█████████▋| 3139960/3237456 [16:56<00:06, 16143.42it/s]


 97%|█████████▋| 3142233/3237456 [16:56<00:05, 17674.11it/s]


 97%|█████████▋| 3144330/3237456 [16:56<00:05, 18522.75it/s]


 97%|█████████▋| 3146745/3237456 [16:56<00:04, 19807.46it/s]


 97%|█████████▋| 3149645/3237456 [16:56<00:04, 21879.78it/s]


 97%|█████████▋| 3152159/3237456 [16:57<00:03, 22718.81it/s]


 97%|█████████▋| 3154520/3237456 [16:57<00:03, 22717.57it/s]


 98%|█████████▊| 3156854/3237456 [16:57<00:03, 22076.79it/s]


 98%|█████████▊| 3159497/3237456 [16:57<00:03, 23218.67it/s]


 98%|█████████▊| 3161980/3237456 [16:57<00:03, 23668.02it/s]


 98%|█████████▊| 3164426/3237456 [16:57<00:03, 23898.00it/s]


 98%|█████████▊| 3166842/3237456 [16:57<00:03, 19349.79

In [ ]:
corpus.dump("politics-filtered-labelled", base_path="/Users/calebchiam/Documents")

In [3]:
corpus = Corpus(filename='/Users/calebchiam/Documents/politics-filtered-labelled')